# CartPole with Policy Gradient Method
In this exercise you will build a policy-gradient based agent that can solve the classic CartPole task where we must balance a pole on a cart for as long as possible. You will specifically be using the OpenAI Gym in order to have a reactive environment that gives you observations (state) and reward with a given action from your model. For those who are not familiar with OpenAI Gym, please check out the short [tutorial](https://gym.openai.com/docs) to cover the basics of the different game environments.

In [1]:
from __future__ import division

import numpy as np
try:
    import cPickle as pickle
except:
    import pickle
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import math

try:
    xrange = xrange
except:
    xrange = range

### Loading the CartPole Environment
If you don't already have the OpenAI gym installed, use  `pip install gym` to grab it.

In [2]:
import gym
env = gym.make('CartPole-v0')

The observation is represented by a 4-dimension vector, i.e. *[position of cart, velocity of cart, angle of pole, rotation rate of pole]*. Note that good general-purpose agents don't need to know the semantics of the observations: they can learn how to map observations to actions to maximize reward without any prior knowledge.

First, we will run the task using random actions. The cart can either move left or right in order to balance the pole. We will randomly choose which direction to move the cart.

## Inline Question 1:
How well do we do with random action? (Hint: not so well.)

Random action strategy does not do well. The reward received in each episode is low (around 10 to 40)

In [ ]:
env.reset()
random_episodes = 0
reward_sum = 0
while random_episodes < 10:
    env.render()
    observation, reward, done, _ = env.step(np.random.randint(0,2))
    reward_sum += reward
    if done:
        random_episodes += 1
        print("Reward for this episode was:",reward_sum)
        reward_sum = 0
        env.reset()

The goal of the task is to achieve a reward of 200 per episode. For every step the agent keeps the pole in the air, the agent recieves a +1 reward. By randomly choosing actions, our reward for each episode is only a couple dozen. Let's make that better with RL!

### Setting up our Neural Network agent
This time we will be using a Policy neural network that takes observations, passes them through a single hidden layer, and then produces a probability of choosing a left/right movement. To learn more about this network, see [Andrej Karpathy's blog on Policy Gradient networks](http://karpathy.github.io/2016/05/31/rl/).

In [3]:
# Hyperparameters
H = 10 # Number of hidden layer neurons
batch_size = 5 # Every how many episodes to do a param update?
learning_rate = 1e-2 # Feel free to play with this to train faster or more stably.
gamma = 0.99 # Discount factor for reward

D = 4 # Input dimensionality

In [14]:
tf.reset_default_graph()

# This defines the network as it goes from taking an observation of the environment to 
# giving a probability of chosing to the action of moving left or right.
observations = tf.placeholder(tf.float32, [None, D], name = "input_x")

W1 = tf.get_variable(
    "W1",
    shape = [D, H],
    initializer = tf.contrib.layers.xavier_initializer()
)

layer1 = tf.nn.relu(tf.matmul(observations, W1))

W2 = tf.get_variable(
    "W2",
    shape = [H, 1],
    initializer = tf.contrib.layers.xavier_initializer()
)

score = tf.matmul(layer1, W2)

# Prediction of action: Probability of taking ACTION NUMBERED 0
probability = tf.nn.sigmoid(score) 

# From here we define the parts of the network needed for learning a good policy.
# tvars = [W1, W2]
tvars = tf.trainable_variables()

# Label action 0, 1
input_y = tf.placeholder(tf.float32, [None, 1], name = "input_y") 

advantages = tf.placeholder(tf.float32, name = "reward_signal")

# The loss function. This sends the weights in the direction of making actions 
# that gave good advantage (reward over time) more likely, and actions that didn't less likely.
loglik = tf.log(
    input_y * (input_y - probability)
    + (1 - input_y) * (input_y + probability)
)

loss = -tf.reduce_mean(loglik * advantages)

# Take the gradient of loss with respect to tvars
newGrads = tf.gradients(loss, tvars)

# Once we have collected a series of gradients from multiple episodes, we apply them.
# We don't just apply gradients after every episode in order to account for noise in the reward signal.
adam = tf.train.AdamOptimizer(learning_rate = learning_rate) # Our optimizer

# Placeholders to send the final gradients through when we update.
W1Grad = tf.placeholder(tf.float32, name = "batch_grad1") 
W2Grad = tf.placeholder(tf.float32, name = "batch_grad2")

batchGrad = [W1Grad, W2Grad]
updateGrads = adam.apply_gradients(zip(batchGrad, tvars))

### Advantage function
This function allows us to weigh the rewards our agent recieves. In the context of the Cart-Pole task, we want actions that kept the pole in the air a long time to have a large reward, and actions that contributed to the pole falling to have a decreased or negative reward. We do this by weighing the rewards from the end of the episode, with actions at the end being seen as negative, since they likely contributed to the pole falling, and the episode ending. Likewise, early actions are seen as more positive, since they weren't responsible for the pole falling.

In [9]:
def discount_rewards(r):
    ################################################################################
    # TODO: Implement the discounted rewards function                              #
    # Return discounted rewards weighed by gamma. Each reward will be replaced     #
    # with a weight reward that involves itself and all the other rewards occuring #
    # after it. The later the reward after it happens, the less effect it has on   #
    # the current rewards's discounted reward                                      #
    # Hint: [r0, r1, r2, ..., r_N] will look someting like:                        #
    #       [(r0 + r1*gamma^1 + ... r_N*gamma^N), (r1 + r2*gamma^1 + ...), ...]    #
    ################################################################################
    
    result = np.zeros_like(r, dtype = np.float32)

    cumm = 0.0        
    for i in xrange(result.shape[0] - 1, -1, -1):
        cumm = cumm * gamma + r[i, 0]
        result[i, 0] = cumm        
    
    return result    
    
    ################################################################################
    #                                 END OF YOUR CODE                             #
    ################################################################################

### Running the Agent and Environment

Here we run the neural network agent, and have it act in the CartPole environment.

In [15]:
xs, hs, dlogps, drs, ys, tfps = [], [], [], [], [], []
running_reward = None
reward_sum = 0
episode_number = 1
total_episodes = 10000
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    rendering = False
    sess.run(init)
    observation = env.reset() # Obtain an initial observation of the environment

    # Reset the gradient placeholder. We will collect gradients in 
    # gradBuffer until we are ready to update our policy network. 
    gradBuffer = sess.run(tvars)
    
    for ix, grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
    
    while episode_number <= total_episodes:        
        # Rendering the environment slows things down, 
        # so let's only look at it once our agent is doing a good job.
        if reward_sum/batch_size > 100 or rendering == True:             
            # env.render()
            rendering = True
            
        # Make sure the observation is in a shape the network can handle.
        x = np.reshape(observation, [1, D])
        
        ################################################################################
        # TODO: Run the policy network and get an action to take                       #
        # Output: action                                                               #
        ################################################################################
        
        probZero = sess.run(probability, feed_dict = {observations: x})[0, 0]
        
        if probZero > 0.5:
            action = 0
        else:
            action = 1
        
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################
             
        xs.append(x) # observation
        y = 1 if action == 0 else 0 # A "fake label"
        ys.append(y)

        ################################################################################
        # TODO: Step the environment and get new measurements                          #
        # Output: observation, reward, done                                            #
        ################################################################################
        
        observation, reward, done, _ = env.step(action)
        
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################

        reward_sum += reward

        # Record reward (has to be done after we call step() to get reward for previous action)
        drs.append(reward) 

        if done: 
            episode_number += 1
            
            # Stack together all inputs, hidden states, action gradients, and rewards for this episode
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            tfp = tfps
            
            # Reset array memory
            xs, hs, dlogps, drs, ys, tfps = [], [], [], [], [], [] 

            # Compute the discounted reward backwards through time
            discounted_epr = discount_rewards(epr)
            
            # Size the rewards to be unit normal (helps control the gradient estimator variance)
            discounted_epr -= np.mean(discounted_epr)
            discounted_epr //= np.std(discounted_epr)
            
            ################################################################################
            # TODO: Run the policy network and get the gradients for this episode          #
            # Output: tGrad                                                                #
            ################################################################################
            
            tGrad = sess.run(newGrads, feed_dict = {
                observations: epx,
                input_y: epy,
                advantages: discounted_epr
            })
            
            ################################################################################
            #                                 END OF YOUR CODE                             #
            ################################################################################
            
            # Save gradients in the gradBuffer
            for ix, grad in enumerate(tGrad):
                gradBuffer[ix] += grad
                
            # If we have completed enough episodes
            if episode_number % batch_size == 0: 
                
                ################################################################################
                # TODO: Update the policy network with our gradients and set gradBuffer to 0   #
                ################################################################################
                
                sess.run(updateGrads, feed_dict = {
                    W1Grad: gradBuffer[0],
                    W2Grad: gradBuffer[1]
                })
                
                for ix, grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0                
                
                ################################################################################
                #                                 END OF YOUR CODE                             #
                ################################################################################
                
                # Give a summary of how well our network is doing for each batch of episodes.
                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                
                print("Average reward for episode %f.  Total average reward %f." % (
                    reward_sum // batch_size,
                    running_reward // batch_size
                ))
                
                if reward_sum // batch_size >= 200: 
                    print("Task solved in", episode_number, "episodes!")
                    break
                    
                reward_sum = 0
            
            observation = env.reset()
        
print(episode_number, "Episodes completed.")

[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
Average reward for episode 7.000000.  Total average reward 7.000000.
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
Average reward for episode 9.000000.  Total average reward 7.000000.
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-

Average reward for episode 11.000000.  Total average reward 7.000000.
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
Average reward for episode 8.000000.  Total average reward 7.000000.
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
Average reward for episode 9.000000.  Total average reward 7.000000.
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 

 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
Average reward for episode 9.000000.  Total average reward 8.000000.
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
Average reward for episode 9.000000.  Total average reward 8.000000.
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 

 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
Average reward for episode 9.000000.  Total average reward 9.000000.
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
Average reward for episode 9.000000.  Total average reward 9.000000.
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [

 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
Average reward for episode 10.000000.  Total average reward 9.000000.
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]]
Average reward for episode 12.000000.  Total average reward 9.000000.
[[ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-2.]
 [-2.]
 [-2.]]
[[ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-

KeyboardInterrupt: 

As you can see, the network not only does much better than random actions, but achieves the goal of 200 points per episode, thus solving the task!